In [60]:
# Red Black Tree implementation in Python 2.7
# Author: Algorithm Tutor
# Tutorial URL: https://algorithmtutor.com/Data-Structures/Tree/Red-Black-Trees/

import sys
from copy import deepcopy

# data structure that represents a node in the tree
class Node():
    def __init__(self, data):
        self.data = data  # holds the key
        self.parent = None #- pointer to the parent
        self.left = None # pointer to left child
        self.right = None # pointer to right child
        self.color = 1 # 1 . Red, 0 . Black
        self.has_been_root = False
        self.birth = None
        self.death = None
    def __str__(self):
        return str(self.data)

# data structure that represents a node in the freezer
class FreezerNode():
    def get_node_val(self,node):
        val = "Null"
        if node:
            val = node.data
        return val
    
    def __init__(self,node,uid,rbt,msg = ""):
        self.data = node.data
        self.uid = uid
        self.children = (self.get_node_val(node.left), self.get_node_val(node.right))
        self.lifespan = (node.birth,rbt.time)
        self.black_depth = rbt.black_depth(node,0)
        self.color = node.color
        self.parent = self.get_node_val(node.parent)
        self.msg = msg
        self.has_been_root = node.has_been_root
        
    def __str__(self):
        return str((str(self.data)+str(self.uid),
                    self.children,
                    f"[{self.lifespan[0]}:{self.lifespan[1]})",
                    self.black_depth,self.color,
                    self.parent,
                    self.msg,
                    ("not root","was root")[self.has_been_root]
                   ))

# data structure that represents the freezer, this should itself be a redblack tree
class Freezer():
    def __init__(self,rbt):
        self.freezer = []
        self.rbt = rbt
    
    def append(self,node,msg=""):
        print("Appending",node,"to freezer at time",self.rbt.time)
        uid = ""
        try:
            max_similar_node_length = max([ len(node1.uid) for node1 in self.freezer if node1.data == node.data ])
            max_similar_node_length = len([ node1 for node1 in self.freezer if node1.data == node.data ])
            uid = "*"*(max_similar_node_length)
        except:
            pass
        frozen_node = FreezerNode(node,uid,self.rbt,msg)
        self.freezer.append(frozen_node)
        
    def __str__(self):
        res = [ str(node) for node in self.freezer ]
        return "\n".join(["Freezer:"] + res)
    
    def __iter__(self):
        return self.freezer.__iter__()


# class RedBlackTree implements the operations in Red Black Tree
class RedBlackTree():
    def __init__(self):
        self.TNULL = Node(0)
        self.TNULL.color = 0
        self.TNULL.left = None
        self.TNULL.right = None
        self.root = self.TNULL
        self.freezer = Freezer(self)
        self.time = 0

    def __pre_order_helper(self, node):
        if node != TNULL:
            sys.stdout.write(node.data + " ")
            self.__pre_order_helper(node.left)
            self.__pre_order_helper(node.right)

    def __in_order_helper(self, node):
        if node != TNULL:
            self.__in_order_helper(node.left)
            sys.stdout.write(node.data + " ")
            self.__in_order_helper(node.right)

    def __post_order_helper(self, node):
        if node != self.TNULL:
            self.__post_order_helper(node.left)
            self.__post_order_helper(node.right)
            sys.stdout.write(node.data + " ")

    def __search_tree_helper(self, node, key):
        if node == self.TNULL or key == node.data:
            return node

        if key < node.data:
            return self.__search_tree_helper(node.left, key)
        return self.__search_tree_helper(node.right, key)

    # fix the rb tree modified by the delete operation
    def __fix_delete(self, x):
        while x != self.root and x.color == 0:
            if x == x.parent.left:
                s = x.parent.right
                if s.color == 1:
                    # case 3.1
                    s.color = 0
                    x.parent.color = 1
                    self.left_rotate(x.parent)
                    s = x.parent.right

                if s.left.color == 0 and s.right.color == 0:
                    # case 3.2
                    s.color = 1
                    x = x.parent
                else:
                    if s.right.color == 0:
                        # case 3.3
                        s.left.color = 0
                        s.color = 1
                        self.right_rotate(s)
                        s = x.parent.right

                    # case 3.4
                    s.color = x.parent.color
                    x.parent.color = 0
                    s.right.color = 0
                    self.left_rotate(x.parent)
                    x = self.root
            else:
                s = x.parent.left
                if s.color == 1:
                    # case 3.1
                    s.color = 0
                    x.parent.color = 1
                    self.right_rotate(x.parent)
                    s = x.parent.left

                if s.left.color == 0 and s.right.color == 0:
                    # case 3.2
                    s.color = 1
                    x = x.parent
                else:
                    if s.left.color == 0:
                        # case 3.3
                        s.right.color = 0
                        s.color = 1
                        self.left_rotate(s)
                        s = x.parent.left 

                    # case 3.4
                    s.color = x.parent.color
                    x.parent.color = 0
                    s.left.color = 0
                    self.right_rotate(x.parent)
                    x = self.root
        x.color = 0

    def __rb_transplant(self, u, v):
        if u.parent == None:
            self.root = v
        elif u == u.parent.left:
            u.parent.left = v
        else:
            u.parent.right = v
        v.parent = u.parent

    def __delete_node_helper(self, node, key):
        # find the node containing key
        z = self.TNULL
        while node != self.TNULL:
            if node.data == key:
                z = node

            if node.data <= key:
                node = node.right
            else:
                node = node.left

        if z == self.TNULL:
            print ("Couldn't find key in the tree")
            return

        y = z
        y_original_color = y.color
        if z.left == self.TNULL:
            x = z.right
            self.__rb_transplant(z, z.right)
        elif (z.right == self.TNULL):
            x = z.left
            self.__rb_transplant(z, z.left)
        else:
            y = self.minimum(z.right)
            y_original_color = y.color
            x = y.right
            if y.parent == z:
                x.parent = y
            else:
                self.__rb_transplant(y, y.right)
                y.right = z.right
                y.right.parent = y

            self.__rb_transplant(z, y)
            y.left = z.left
            y.left.parent = y
            y.color = z.color
        if y_original_color == 0:
            self.__fix_delete(x)
    
    # fix the red-black tree
    def  __fix_insert(self, k):
        while k.parent.color == 1:
            if k.parent == k.parent.parent.right:
                u = k.parent.parent.left # uncle
                if u.color == 1:
                    # case 3.1
                    u.color = 0
                    k.parent.color = 0
                    k.parent.parent.color = 1
                    k = k.parent.parent
                else:
                    if k == k.parent.left:
                        # case 3.2.2
                        k = k.parent
                        self.right_rotate(k)
                    # case 3.2.1
                    k.parent.color = 0
                    k.parent.parent.color = 1
                    self.left_rotate(k.parent.parent)
            else:
                u = k.parent.parent.right # uncle

                if u.color == 1:
                    # mirror case 3.1
                    u.color = 0
                    k.parent.color = 0
                    k.parent.parent.color = 1
                    k = k.parent.parent 
                else:
                    if k == k.parent.right:
                        # mirror case 3.2.2
                        k = k.parent
                        self.left_rotate(k)
                    # mirror case 3.2.1
                    k.parent.color = 0
                    k.parent.parent.color = 1
                    self.right_rotate(k.parent.parent)
            if k == self.root:
                break
        self.root.color = 0

    def __print_helper(self, node, indent, last):
        # print the tree structure on the screen
        if node != self.TNULL:
            sys.stdout.write(indent)
            if last:
                sys.stdout.write("R----")
                indent += "     "
            else:
                sys.stdout.write("L----")
                indent += "|    "

            s_color = "RED" if node.color == 1 else "BLACK"
            print(str(node.data) + "(" + s_color + ")")
            self.__print_helper(node.left, indent, False)
            self.__print_helper(node.right, indent, True)
    
    # Pre-Order traversal
    # Node.Left Subtree.Right Subtree
    def preorder(self):
        self.__pre_order_helper(self.root)

    # In-Order traversal
    # left Subtree . Node . Right Subtree
    def inorder(self):
        self.__in_order_helper(self.root)

    # Post-Order traversal
    # Left Subtree . Right Subtree . Node
    def postorder(self):
        self.__post_order_helper(self.root)

    # search the tree for the key k
    # and return the corresponding node
    def searchTree(self, k):
        return self.__search_tree_helper(self.root, k)

    # find the node with the minimum key
    def minimum(self, node):
        while node.left != self.TNULL:
            node = node.left
        return node

    # find the node with the maximum key
    def maximum(self, node):
        while node.right != self.TNULL:
            node = node.right
        return node

    # find the successor of a given node
    def successor(self, x):
        # if the right subtree is not None,
        # the successor is the leftmost node in the
        # right subtree
        if x.right != self.TNULL:
            return self.minimum(x.right)

        # else it is the lowest ancestor of x whose
        # left child is also an ancestor of x.
        y = x.parent
        while y != self.TNULL and x == y.right:
            x = y
            y = y.parent
        return y

    # find the predecessor of a given node
    def predecessor(self,  x):
        # if the left subtree is not None,
        # the predecessor is the rightmost node in the 
        # left subtree
        if (x.left != self.TNULL):
            return self.maximum(x.left)

        y = x.parent
        while y != self.TNULL and x == y.left:
            x = y
            y = y.parent

        return y

    # rotate left at node x
    def left_rotate(self, x):
        y = x.right
        # before rotation begins, we add x, y and x.parent to the freezer
        if x.parent:
            self.freezer.append(x.parent,"left rotate parent")
        self.freezer.append(x,"left rotate")
        self.freezer.append(y,"left rotate")
        x.birth = self.time
        y.birth = self.time
        x.right = y.left
        if y.left != self.TNULL:
            y.left.parent = x

        y.parent = x.parent
        if x.parent == None:
            self.root = y
        elif x == x.parent.left:
            x.parent.left = y
        else:
            x.parent.right = y
        y.left = x
        x.parent = y

    # rotate right at node x
    def right_rotate(self, x):
        y = x.left
        # before rotation begins, we add x, y and x.parent to the freezer
        if x.parent:
            self.freezer.append(x.parent,"right rotate parent")
        self.freezer.append(x,"right rotate")
        self.freezer.append(y,"right rotate")
        x.birth = self.time
        y.birth = self.time
        x.left = y.right
        if y.right != self.TNULL:
            y.right.parent = x

        y.parent = x.parent
        if x.parent == None:
            self.root = y
        elif x == x.parent.right:
            x.parent.right = y
        else:
            x.parent.left = y
        y.right = x
        x.parent = y

    # insert the key to the tree in its appropriate position
    # and fix the tree
    def insert(self, key, time):
        # Ordinary Binary Search Insertion
        node = Node(key)
        node.parent = None
        node.data = key
        node.left = self.TNULL
        node.right = self.TNULL
        node.color = 1 # new node must be red
        node.birth = time
        self.time = time

        y = None
        x = self.root

        while x != self.TNULL:
            y = x
            if node.data < x.data:
                x = x.left
            else:
                x = x.right

        # y is parent of x
        node.parent = y
        if y == None:
            self.root = node
        elif node.data < y.data:
            y.left = node
        else:
            y.right = node
        
        # if new node is a root node, simply return
        if node.parent == None:
            node.color = 0
            if self.root:
                self.root.has_been_root = True
            return

        # if the grandparent is None, simply return
        if node.parent.parent == None:
            if self.root:
                self.root.has_been_root = True
            return

        # Fix the tree
        self.__fix_insert(node)
        
        if self.root:
            self.root.has_been_root = True
        
        

    def get_root(self):
        return self.root
    
    # avoids null pointer
    def get_node_val(self,node):
        val = "Null"
        if node:
            val = node.data
        return val
    
    # calculate the black depth of a node in the tree
    def black_depth(self,node,depth):
        if node == None:
            return depth
        else:
            if node.color == 1:
                d = 0
            else:
                d = 1
            #d = depth + (node.color ^ 1) # red = 1, so red xor 1 = 0 and black xor 1 = 1
            return self.black_depth(node.left,depth + d)

    # delete the node from the tree
    def delete_node(self, data, time):
        self.time = time
        node_to_delete = self.searchTree(data)
        node_to_delete.death = time
        left = self.get_node_val(node_to_delete.left)
        right = self.get_node_val(node_to_delete.right)
        self.freezer.append(node_to_delete,"deletion")
        #self.freezer.append((node_to_delete.data,\
        #                     (left,right),\
        #                     f"[{node_to_delete.birth}:{node_to_delete.death}]",\
        #                     self.black_depth(node_to_delete,0)))
        self.__delete_node_helper(self.root, data)
        if self.root:
            self.root.has_been_root = True
        

    # print the tree structure on the screen
    def pretty_print(self):
        self.__print_helper(self.root, "", True)

def big_test():
    bst = RedBlackTree()
    x = 0
    bst.insert(7,x)
    bst.insert(11,x)
    print(f"{x=}")
    x += 1
    bst.pretty_print()
    bst.delete_node(7,x)
    bst.pretty_print()
    bst.insert(1,x)
    bst.pretty_print()
    bst.insert(8,x)
    print(f"{x=}")
    x += 1
    bst.pretty_print()
    bst.delete_node(8,x)
    bst.delete_node(11,x)
    bst.insert(10,x)
    bst.insert(4,x)
    print(f"{x=}")
    x += 1
    bst.pretty_print()
    bst.delete_node(1,x)
    bst.insert(2,x)
    print(f"{x=}")
    x += 1
    bst.pretty_print()
    bst.insert(12,x)
    print(f"{x=}")
    x += 1
    bst.pretty_print()
    bst.delete_node(12,x)
    bst.delete_node(10,x)
    bst.insert(13,x)
    bst.insert(9,x)
    print(f"{x=}")
    x += 1
    bst.pretty_print()
    bst.delete_node(4,x)
    bst.delete_node(2,x)
    bst.insert(5,x)
    bst.insert(3,x)
    print(f"{x=}")
    x += 1
    bst.pretty_print()
    bst.delete_node(5,x)
    bst.delete_node(9,x)
    bst.insert(6,x)
    print(f"{x=}")
    x += 1
    bst.pretty_print()
    bst.delete_node(3,x)
    print(f"{x=}")
    x += 1
    bst.pretty_print()
    bst.delete_node(13,x)
    bst.delete_node(6,x)
    print("x=9 and done")
    bst.pretty_print()
    print("Printing freezer")
    print("Format: (value, (left,right), [birth:death], black depth)")
    for node in bst.freezer:
        print(node)
    print("\n\nPrinting sorted freezer") #sort by black_depth then by colour (red before black)
    freeze = sorted(bst.freezer,key = lambda x: (x.black_depth,-x.color))
    for node in freeze:
        print(node)

def small_test():
    rbt = RedBlackTree()
    rbt.insert(1,1)
    rbt.pretty_print()
    rbt.insert(2,2)
    rbt.pretty_print()
    rbt.insert(3,3)
    rbt.pretty_print()
    rbt.insert(4,4)
    rbt.pretty_print()
    rbt.delete_node(3,5)
    rbt.pretty_print()
    rbt.delete_node(2,6)
    rbt.pretty_print()
    rbt.delete_node(1,7)
    rbt.pretty_print()
    rbt.delete_node(4,8)
    rbt.pretty_print()
    freeze = sorted(rbt.freezer,key = lambda x: (x.black_depth,-x.color))
    for node in freeze:
        print(node)

small_test()

R----1(BLACK)
R----1(BLACK)
     R----2(RED)
Appending 1 to freezer at time 3
Appending 2 to freezer at time 3
R----2(BLACK)
     L----1(RED)
     R----3(RED)
R----2(BLACK)
     L----1(BLACK)
     R----3(BLACK)
          R----4(RED)
Appending 3 to freezer at time 5
R----2(BLACK)
     L----1(BLACK)
     R----4(BLACK)
Appending 2 to freezer at time 6
R----4(BLACK)
     L----1(RED)
Appending 1 to freezer at time 7
R----4(BLACK)
Appending 4 to freezer at time 8
('1', (0, 2), '[1:3)', 1, 1, 'Null', 'left rotate', 'was root')
('1*', (0, 0), '[3:7)', 1, 1, 4, 'deletion', 'was root')
('2', (0, 3), '[2:3)', 2, 0, 1, 'left rotate', 'not root')
('3', (0, 4), '[3:5)', 2, 0, 2, 'deletion', 'not root')
('4', (0, 0), '[4:8)', 2, 0, 'Null', 'deletion', 'was root')
('2*', (1, 4), '[3:6)', 3, 0, 'Null', 'deletion', 'was root')


In [2]:
class A():
    def __init__(self):
        self.note = "a"
        
class B():
    def __init__(self):
        self.note = "b"
        
class C():
    def __init__(self,node):
        self.note = node().note
        
c = C(B)
print(c.note)

b
